## 安裝套件

In [ ]:
!pip install pandarallel
!pip install pyahocorasick
!pip install --upgrade pip
!pip install spacy==3.0.*

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

from pandarallel import pandarallel
pandarallel.initialize()

import spacy
# spacy.require_gpu()
from spacy.training import Example
from spacy.util import minibatch
import random

import ahocorasick

## 讀取資料

In [ ]:
df = pd.read_csv("../input/scl-2021-ds/train.csv")
df.set_index("id", inplace=True)
df['POI'] = np.nan
df['street'] = np.nan

def extract_entities(row):
    extracted = row['POI/street'].split("/")
    
    if len(extracted) == 2:
        poi, street = extracted
        if poi.strip() != '':
            row['POI'] = poi
        
        if street.strip() != '':
            row['street'] = street
        
    return row

df = df.parallel_apply(extract_entities, axis=1)
nlp = spacy.blank('id')  # create blank Language class

## 處理資料

In [ ]:
from copy import deepcopy

def _build_aho(words):
    aho = ahocorasick.Automaton()
    for idx, key in enumerate(words):
        aho.add_word(key, (idx, key))

    return aho

## 格式化data
def format_data(text, poi, street):
    entities = []
    _text = deepcopy(text)
    
    ## 處理poi
    if isinstance(poi, str):
        aho = _build_aho([poi])
        aho.make_automaton()
        latest_char_idx = 0
        
        for end, (_, word) in aho.iter(_text):
            start = end - len(word) + 1
            if start < latest_char_idx:
                continue

            entities.append((start, end + 1, 'POI'))
            _text = _text.replace(word, " " * len(word))
            latest_char_idx = end + 1
    
    ## 處理street
    if isinstance(street, str):
        aho = _build_aho([street])
        aho.make_automaton()
        latest_char_idx = 0

        for end, (_, word) in aho.iter(_text):
            start = end - len(word) + 1
            if start < latest_char_idx:
                continue

            entities.append((start, end + 1, 'STREET'))
            latest_char_idx = end + 1
    
    return Example.from_dict(nlp.make_doc(text), {"entities": entities})

In [ ]:
print("Preparing Spacy examples...")

examples = []
for idx in df.index:
    try:
        row = df.loc[idx]
        example = format_data(row['raw_address'], row['POI'], row['street'])
        examples.append(example)
    except Exception as e:
        print(idx)
        print("-" * 50)
        print(e)
        break

## 訓練

In [ ]:
def train_spacy(nlp, examples, iterations):
    
    TRAIN_DATA = examples
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.add_pipe('ner', last=True)

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Starting iteration " + str(itn))
            random.shuffle(examples)
            batches = minibatch(examples, 1000)
            losses = {}
            for batch in batches:
                nlp.update(
                    batch,
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp

In [ ]:
train = examples

In [ ]:
%%time
ner_nlp = train_spacy(nlp, train, 60)

## 看預測結果

In [ ]:
for idx, row in df.iloc[100:110].iterrows():
    print(f"address: {row['raw_address']}")
    print(f"expected poi: {row['POI']}")
    print(f"expected street: {row['street']}")
    print()
    
    doc = ner_nlp(row['raw_address'])
    for ent in doc.ents:
        print(ent.text, "-", ent.label_)

    print("-" * 50)

## 將test資料預測

In [ ]:
df_test = pd.read_csv("../input/scl-2021-ds/test.csv")
df_test.set_index("id", inplace=True)

In [ ]:
submission = []
for idx, row in df_test.iloc[:].iterrows():
    doc = ner_nlp(row['raw_address'])
    tmp = {'id': idx}
    for ent in doc.ents:
        tmp[ent.label_] = ent.text
    submission.append(tmp)
    


In [ ]:
submission = pd.DataFrame(submission)
submission['POI'] = submission['POI'].replace(np.nan, '')
submission['STREET'] = submission['STREET'].replace(np.nan, '')
submission['POI/street'] = submission['POI'] + '/' + submission['STREET'] 

In [ ]:
pd.DataFrame({'id':submission['id'],'POI/street':submission['POI/street']}).to_csv('submission.csv', header=True, index=False)